# AnimationKit AI - Synthetic Video Upscaling & Interpolation Tool

**ALPHA 2.5: Frostbite Revival (Released 12/23/21)**

Changelog:

- [ UI ] Chained design. All steps link to one another! Use the master override toggles to skip processes.
- [ Upscaling ] GFPGAN face-enhance toggle (embedded in Real-ESRGAN)
- [ Interpolation ] Practical-RIFE implementation; 4.0 model
- [ File mgmt ] File overwrite protection, verification of completion
- [ Misc ] Forked essential dependencies for long-term backwards compatibility; if needed.

---

Main branch and any potential updates can be found here: https://github.com/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN

---
#Credits

Motion smoothing utilizes [Practical-RIFE](https://github.com/hzwer/Practical-RIFE) 4.0 model.

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833). A demo can be found at https://raw.githubusercontent.com/xinntao/Real-ESRGAN

In [ ]:
# @title AnimationKit AI License(s): { form-width: "300px" }
# @markdown Practical-RIFE uses the MIT License.

# @markdown Real-ESRGAN uses the BSD 3-Clause License.

# @markdown *To view the included licenses, click "Show code."* 
# -------------------------------------------------
# Below is the MIT license for RIFE & Practical-RIFE
# ------------------------------------------------

# MIT License

# Copyright (c) 2021 hzwer

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# -------------------------------------------------
# Below is the BSD3-Clause License for Real-ESRGAN.
# ------------------------------------------------

# BSD 3-Clause License

# Copyright (c) 2021, Xintao Wang
# All rights reserved.

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer.

# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.

# 3. Neither the name of the copyright holder nor the names of its
#    contributors may be used to endorse or promote products derived from
#    this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.




In [ ]:
#@markdown ##Check GPU
#@markdown ***This Notebook supports "Run All. Once everything is set to your liking, you can do `Ctrl+F9`!***
import torch
# Check the GPU status
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

enable_error_checking = False #saves ram
if enable_error_checking:
  !nvidia-smi
else:
  !nvidia-smi
  !nvidia-smi -i 0 -e 0
#@markdown *Special thanks to sportracer48 and his Discord channel. If you want to make AI animations, PyTTI is king.* https://www.patreon.com/sportsracer48

# Mount Google Drive, Install Libraries, Load Functions


In [ ]:
#@title  { form-width: "300px" }
#@markdown *Note: This project has been tested around Google Drive and may not yet support non-Drive configurations.*
#https://unix.stackexchange.com/questions/70963/difference-between-2-2-dev-null-dev-null-and-dev-null-21
import os, sys, re, gc  #>/dev/null
gc.collect()
from multiprocessing import Process

def Dir_check(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("ERROR: "+a+" does not exist!")

def Dir_make(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("Creating "+a+" ...")
    !mkdir -p $a

def File_check(a):
  import os.path
  from os import path
  if not path.isfile(a):
    return False
  else:
    return True

# def need_file(a):
#   import os.path
#   from os import path
#   if not path.isfile(a):
#     print("Creating"+a+"...")
#     !mkdir -p $a



# def File_check(a):
#   import os.path
#   from os import path
#   if not path.isfile(a):
#     print("ERROR: "+a+" does not exist!")


class AnKit:
  #def __init__(self):
  #  pass
  import os.path
  from os import path
  #video tools
  def video_sortFrames(sourceframes, destframes):  #takes frames from input folder, moves to init_frame_storage
    Dir_check(sourceframes)
    Dir_make(destframes)
    %cd $sourceframes
    print("Copying frames to "+destframes+" for processing...")
    !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $destframes
    %cd $destframes
    #!find -maxdepth 1 -name '*.png' -print0 | xargs -0 'mv "$0" "${0##*_}"' 
    #!find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    !find . -type f -name "*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    #print("Padding filenames in "+destframes+".")
    !rename 's/\d+/sprintf("%05d",$&)/e' *  #adds padding to numbers
    print("Finished copying frames to "+destframes+".")
  def video_splitFrames(sourcefile, destframes):
    File_check(sourcefile)
    Dir_make(destframes)
    !ffmpeg -y -r 1 -i $sourcefile -r 1 $destframes/frame%05d.png
  def frames2video(sourceframes,fps,outputmp4):
    Dir_check(sourceframes)
    %cd $sourceframes
    !ffmpeg -framerate $fps -pattern_type glob -i '*.png' -y $outputmp4
    
    #!ffmpeg -framerate $fps -pattern_type glob -i '*.png' -vcodec hevc_nvenc -y $outputmp4
    #this vcodec doesnt work with a100; nixing for now

  def video_runUpscale(esrgan,mname,scale,input,output):
    Dir_check(input)
    Dir_make(output)
    %cd $esrgan
    !python inference_realesrgan.py --model_name $mname --input $input --output $output
  def RIFE_video(fps,exp,input,scale,output):
    %cd /content/Practical-RIFE
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --video=$input --scale $scale --output=$output
  def RIFE_frames(fps,exp,input,scale,output):  #not currently working right
    %cd /content/Practical-RIFE
    input = input + '/'
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --img=$input --scale $scale --output=$output
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = int(a_string)
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(measured_duration,_target_length_seconds): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = _target_fps * _target_length_seconds
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x
  #installation for Colab (untested on other platforms)
  #
  #
  def install_ESRGAN():
    print("Installing libraries for Real-ESRGAN upscaling.")
    #!git clone https://github.com/xinntao/Real-ESRGAN.git &> /dev/null #using own fork for longevity
    !git clone https://github.com/sadnow/Real-ESRGAN_AnimationKit Real-ESRGAN &> /dev/null #using own fork for longevity
    %cd Real-ESRGAN
    !pip -q install basicsr
    !pip -q install facexlib
    !pip -q install gfpgan
    !pip -q install -r requirements.txt   &> /dev/null
    !python setup.py develop              &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models              &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models   &> /dev/null
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models              &> /dev/null
    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    #
  def install_RIFE():
    %cd /content/
    print("Installing libraries for RIFE motion smoothing.")
    #!git clone https://github.com/hzwer/Practical-RIFE Practical-RIFE &> /dev/null  #using my own archive for longevity
    !git clone https://github.com/sadnow/Practical-RIFE_AnimationKit Practical-RIFE &> /dev/null  #using my own archive for longevity
    #!gdown --id 1O5KfS3KzZCY3imeCr2LCsntLhutKuAqj &> /dev/null  #DEPRECATED 3.8 MODEL
    !curl -L https://github.com/chigozienri/AnimationKit-AI/blob/main/RIFE_trained_model_v4.0.zip?raw=true > RIFE_trained_model_v4.0.zip  #NEWER 4.0 MODEL
    #!7z e RIFE_trained_model_v3.8.zip &> /dev/null
    !7z e RIFE_trained_model_v4.0.zip &> /dev/null
    !mkdir /content/Practical-RIFE/train_log  &> /dev/null
    !mv *.py /content/Practical-RIFE/train_log/ &> /dev/null
    !mv *.pkl /content/Practical-RIFE/train_log/  &> /dev/null
    %cd /content/Practical-RIFE/
    #!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc   &> /dev/null  #useless - mp4 demo 
    !pip3 install -r requirements.txt &> /dev/null
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    #  
    %cd /content/
  def depcrecated_installOldRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
    !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
    !7z e RIFE_trained_model_HDv2.zip
    !mkdir /content/RIFE/train_log
    !mv *.pkl /content/RIFE/train_log/
    %cd /content/RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
    !pip3 install -r requirements.txt
    print("Done.")
    print("Finsihed Installing libraries for RIFE motion smoothing.")
  %cd /content/
  
  def install_3dinpainting(): #untested as a function
    !pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
    !pip3 install opencv-python==4.2.0.32
    !pip3 install vispy==0.6.4
    !pip3 install moviepy==1.0.2
    !pip3 install transforms3d==0.3.1
    !pip3 install networkx==2.3
    !sudo apt install sed
    %cd /content/
    !git clone https://github.com/sadnow/3d-photo-inpainting.git
    %cd 3d-photo-inpainting
    !sh download.sh
    !sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

  def run_3dinpainting(): #untested as a function
    !mkdir image
    !mkdir video
    %cd image
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    %cd ..
    !python main.py --config argument.yml
###


###############################################################

#
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
AnKit.install_ESRGAN()
AnKit.install_RIFE()
!pip install lpips datetime -q  #for overwrite protection
####################################################

In [ ]:
#@title  { form-width: "300px" }
#@markdown # AnimationKit AI
#@markdown >*This cell combines Real-ESRGAN upscaling, RIFE smoothing/interpolation, and h265 compression.*
#@markdown ---
#@markdown ##Input settings
_import_mp4_file = False #@param {type:"boolean"}
_imported_mp4_file='/content/drive/MyDrive/VQLIPSE/videos/filespace-dinner.mp4' #@param {type:"string"}
#@markdown >If you want to process from static images, uncheck `_import_mp4_file`
_import_frame_folder='/content/drive/MyDrive/pytti_test/images_out/zelda3/' #@param {type:"string"}
#@markdown Note: *`_import_frame_folder` is for importing individual frames from your projects. Will be ignored if `_import_mp4_file` is checked.*

#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Output settings
_output_folder = '/content/drive/MyDrive/AI/AnimationKit/videos/' #@param {type:"string"}
_output_filename = 'zelda3_AnKit.mp4' #@param{type:"string"}
_target_fps=60#@param {type:"integer"}
_target_length_seconds=30#@param {type:"integer"}
#@markdown ---
_upscale_model_name='RealESRGAN_x4plus_anime_6B' #@param ['RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
_constant_quality=32#@param {type:"slider", min:20, max:50, step:1}
_face_enhance_gfpgan=True #@param{type:"boolean"}
_skip_static_frames = False #deprecated
# keep_mp4_audio = False #@param {type: "boolean"}
#@markdown Default `_constant_quality` is `30`. Higher values = lower filesize, lower quality


#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Performance
#input_path='/content/drive/MyDrive/pytti_test/videos/zelda4.mp4' #@param {type:"string"}
_target_scale_RIFE='1.0'#@param ['0.25','0.5','1.0','2.0']{type:"string"}
_half_precision_realesrgan=False #@param{type:"boolean"}
#length_multiplier=3#@param {type:"integer"}
#@markdown *These options can speed up processing at the costs of quality.*
#@markdown If you're seeing weird warping in your outputs, try increasing or decreasing `_target_scale_RIFE`.
#@markdown 
#---------------------------------------------------------------#


#compress_path='/content/drive/MyDrive/pytti_test/videos/zelda4_16X_180fps.mp4' #@param {type:"string"}
#outputStr = '_tblend2.mp4' #@param {type:"string"}
enable_tblend = False #deprecated
#deflicker_on = False #@param {type:"boolean"}
#deflicker_avg_frames=76#@param {type:"slider", min:2, max:129, step:1}
#mpdecimate_on = False #@param {type:"boolean"}
#minterpolate_on = False #@param {type:"boolean"}
#minterpolate_fps=60#@param {type:"integer"}
#tblend_on = False #@param {type:"boolean"}
#tmix_on = True #@param {type:"boolean"}
tblend_framestep_value = "2"  #deprecated
##@markdown Set `deflicker_avg_frames` filter size in frames. FFmpeg's default is 5. I haven't tested too much but I would try 100.

#---------------------------------------------------------------#
#@markdown ---
#@markdown ##Master Overrides
#@markdown If you want to skip certain processes, this is the master toggle. All related settings above will be ignored, so no need to change them.
#skip_cleanup = True #@param{type:"boolean"}
_skip_upscaling = False #@param{type:"boolean"}
_skip_RIFEsmoothing = False #@param{type:"boolean"}
##@markdown `skip_cleanup` will be automatically enabled if you pick either of these options.
#---------------------------------------------------------------#
skip_cleanup = False
if _skip_upscaling:
  skip_cleanup = True
if _skip_RIFEsmoothing:
  skip_cleanup = True

###################################################################
#Conditional passing of flags

#---------------------------------------------------------------#
# check if the message ends with .mp4
contains_extension = (_output_filename.endswith('.mp4'))
if not contains_extension:
  _output_filename = _output_filename + '.mp4'
output_fullpathname = _output_folder + _output_filename

contains_slash = (_output_folder.endswith('/'))
if not contains_slash:
  _output_folder = _output_folder + '/'
#---------------------------------------------------------------#

if enable_tblend: #DEPRECATED
  visual_effects = '-vf tblend=all_mode=average,framestep=' + tblend_framestep_value
else:
  visual_effects = ''
#---------------------------------------------------------------#

added_params = ""
if _half_precision_realesrgan:
  added_params = added_params + " --half"
if _face_enhance_gfpgan: 
  added_params = added_params + " --face_enhance"

if _skip_static_frames: #DEPCRECATED
  _target_scale_RIFE = _target_scale_RIFE + ' --skip'

%cd /content/
#cleanup
if not skip_cleanup:
  print("\n Cleaning up from last run...\n")
  !rm -rf "/content/Real-ESRGAN/results"
  !rm -rf '/content/frames_storage/init_frame_storage'
  !rm -rf '/content/frames_storage/upscaled_frame_storage'

###############################################################
#P1 REAL-ESRGAN UPSCALING

print("\n ------------------------------------\n")
print("\n Beginning Real-ESRGAN Upscaling phase...\n")

if _import_mp4_file:
  from pathlib import Path
  #---splitframes------------------------------------------------
  print("\n Starting splitFrames... \n")
  File_check(_imported_mp4_file)
  Dir_make('/content/frames_storage/init_frame_storage')
  !ffmpeg -y -r 1 -i $_imported_mp4_file -r 1 '/content/frames_storage/init_frame_storage'/frame%05d.png
  print("\n Completed splitFrames... \n")
  #---end------------------------------------------------

print("\n Beginning sortFrames... ")
if not _import_mp4_file:
  Dir_check(_import_frame_folder)
  %cd $_import_frame_folder
Dir_make('/content/frames_storage/init_frame_storage')

#---sortframes------------------------------------------------
print("\n Copying frames to "+'/content/frames_storage/init_frame_storage'+" for processing...\n")
!find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t '/content/frames_storage/init_frame_storage'
%cd '/content/frames_storage/init_frame_storage'
!find . -type f -name "*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbered (junk files) from new dir
!rename 's/\d+/sprintf("%05d",$&)/e' *  #adds padding to numbers
print("\n Finished copying frames to "+'/content/frames_storage/init_frame_storage'+".\n")
print("Completed sortFrames... \n")
#---end-------------------------------------------------

%cd /content/

#---runUpscale------------------------------------------------
if _skip_upscaling:
  print("NOTICE: _skip_upscaling is checked. Skipping Real-ESRGAN upscaling...\n")
else:
  Dir_check('/content/frames_storage/init_frame_storage')
  Dir_make('/content/frames_storage/upscaled_frame_storage')
  %cd '/content/Real-ESRGAN/'
  !python inference_realesrgan.py --model_name $_upscale_model_name --input '/content/frames_storage/init_frame_storage' --output '/content/frames_storage/upscaled_frame_storage'$added_params
#---end------------------------------------------------

#---frames2video---------------------------------------------
print("\n NOTICE: Running frames2video \n")
Dir_check('/content/frames_storage/upscaled_frame_storage')
%cd '/content/frames_storage/upscaled_frame_storage'
print("\n Creating wip_file from frames... \n")
!ffmpeg -framerate $_target_fps -pattern_type glob -i '*.png' -y '/content/frames2video.mp4'  
#this marks the creation of wip_file

print("\n NOTICE: Finished running frames2video \n")
print("End of upscaling phase.\n")
#---end------------------------------------------------

#END OF UPSCALING PHASE P1
#END OF UPSCALING PHASE

###############################################################################
###############################################################################

#P2 RIFE MOTION SMOOTHING
#P2 RIFE MOTION SMOOTHING


print("\n ------------------------------------\n")
print("\n Beginning RIFE motion smoothing phase... \n")

%cd /content/Practical-RIFE/

#----------------------------
if _import_mp4_file:
  measured_fps = AnKit.detect_fps(_imported_mp4_file)
  print("\n NOTICE: Detected average FPS of ",_imported_mp4_file," is ",measured_fps)
  measured_duration = AnKit.detect_duration(_imported_mp4_file)
else: 
  measured_fps = AnKit.detect_fps('/content/frames2video.mp4')
  print("\n NOTICE: Detected average FPS of ","/content/frames2video.mp4"," is ",measured_fps)
  measured_duration = AnKit.detect_duration('/content/frames2video.mp4')

print("\n NOTICE: Detected duration INTEGER (in seconds) is ",measured_duration)

if measured_duration < 1: #failsafe
  print("\n NOTICE: Your input appears to be less than one second... \n")
  measured_duration = 1

exp_value = AnKit.exp_calc(measured_duration,_target_length_seconds)


#print("\n NOTICE: Target duration currently rounds to the closest --exp RIFE can handle. \n")

if (exp_value < 0.5):
  _skip_RIFEsmoothing = True
  print("\n NOTICE: Your _target_fps doesn't necessitate RIFE motion smoothing. Skipping RIFE...\n")

if _skip_RIFEsmoothing:
  print("\n NOTICE: Skipping RIFE motion smoothing...\n")
else:
  #---RUN RIFE------------------------------------
  print("\n NOTICE: Running RIFE... \n")
  %cd /content/Practical-RIFE
  !python3 /content/Practical-RIFE/inference_video.py --fps=$_target_fps --exp=$exp_value --video='/content/frames2video.mp4' --scale=$_target_scale_RIFE --output='/content/RIFE_frames2video.mp4'
  
  #---END--------------------------------------
#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
#END OF MOTION SMOOTHING PHASE
print("\n End of RIFE interpolation phase.\n")

#########################################################################
#P3 h265 compression
print("\n ------------------------------------\n")
print("\n NOTICE: Beginning h265 compression phase...\n")
%cd /content/
Dir_make(_output_folder)

#---File overwrite protection---
overwrite_protection = File_check(output_fullpathname)
if overwrite_protection:  #add datetime to end of file
  from datetime import datetime
  current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
  output_fullpathname = output_fullpathname + '_' + current_time + '.mp4'
  print("NOTICE: Overwrite protection has renamed your output file as ",output_fullpathname," \n")
#---end--------------------

if _skip_RIFEsmoothing:  
  !ffmpeg -i '/content/frames2video.mp4' $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
else:
  !ffmpeg -i '/content/RIFE_frames2video.mp4' $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname

print("\n End of h265 compression phase.\n")
file_verified = File_check(output_fullpathname)
if file_verified:
  print("\n NOTICE: Finished! Your final file is saved as ",output_fullpathname)
else:
  print ("\n ERROR: Cannot verify that your file completed processing! Check /content/ for mp4 files to see what part of the process went wrong.")


This Notebook has been designed so you can "Run All". [**Ctrl+F9**]